## Задача про поиск похожих слов по эмбеддингам

На основе word2vec/fasttext слоя Embedding реализовать метод поиска ближайших твитов (на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)

### Подготовка корпуса данных

In [ ]:
#!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
#!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

In [1]:
import pandas as pd
import numpy as np

# Загружаем данные из CSV-файлов
positive_df = pd.read_csv('positive.csv', sep=';')
negative_df = pd.read_csv('negative.csv', sep=';')

# Объединяем их в одну выборку
combined_df = pd.concat([positive_df, negative_df], axis=0)

# Сохраняем объединенный датасет в новый CSV-файл, если необходимо
combined_df.to_csv('combined_dataset.csv', index=False)

#combined_df.info()

In [3]:
# Создание полей для исходного датафрейма
Field_df = pd.DataFrame()
field_template = "Field_{}"

# Задаем количество полей, которые нужно создать
num_fields = combined_df.shape[1]

# Создаем столбцы на основе шаблона
for i in range(1, num_fields + 1):
    field_name = field_template.format(i)
    Field_df[field_name] = None  # Здесь можно указать начальные значения для столбцов

Field_df = list(Field_df)

combined_df.columns = Field_df
combined_df = combined_df.astype('str')

# Берем только твиты и лейблы
combined_df = combined_df[['Field_4', 'Field_5']]

new_column_names = ['tweet', 'label']
combined_df.columns = new_column_names
combined_df.head()

,tweet,label
0,"Да, все-таки он немного похож на него. Но мой ...",1.0
1,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1.0
2,"RT @digger2912: ""Кто то в углу сидит и погибае...",1.0
3,@irina_dyshkant Вот что значит страшилка :D\nН...,1.0
4,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1.0


### Предобработка текста

In [4]:
selected_columns = combined_df
selected_columns.head()

,tweet,label
0,"Да, все-таки он немного похож на него. Но мой ...",1.0
1,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1.0
2,"RT @digger2912: ""Кто то в углу сидит и погибае...",1.0
3,@irina_dyshkant Вот что значит страшилка :D\nН...,1.0
4,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1.0


In [5]:
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re

# Инициализируйте библиотеки nltk и pymorphy2
nltk.download('stopwords')
nltk.download('punkt')
morph = MorphAnalyzer()
stop_words = set(stopwords.words("russian"))

# Пример твита
tweet = "Написал код для обработки текстовых данных в Python! #python #NLP"

# Функция для предобработки текста
def preprocess_tweet(tweet):
    if isinstance(tweet, str):
        # Токенизация: разделение текста на слова или токены
        words = nltk.word_tokenize(tweet)
        
        # Приведение к нижнему регистру
        words = [word.lower() for word in words]
        
        # Удаление знаков препинания и цифр
        words = [re.sub(r'[^а-яА-Яa-zA-Z]', '', word) for word in words]
        
        # Удаление стоп-слов
        words = [word for word in words if word not in stop_words]
        
        # Лемматизация: приведение слов к их базовой форме
        words = [morph.parse(word)[0].normal_form for word in words]
        
        return ' '.join(words)

    else:
        return words

# Применение предобработки к твиту
preprocessed_tweet = preprocess_tweet(tweet)
print(preprocessed_tweet)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vddav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vddav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


написать код обработка текстовый данные python   python  nlp


In [6]:
selected_columns = selected_columns.dropna(subset=['tweet'])
selected_columns['tweet'] = selected_columns['tweet'].astype(str)
selected_columns['tweet'] = selected_columns['tweet'].apply(preprocess_tweet)
#selected_columns.head(1)

### Обучение модели Word Embeddings

In [7]:
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import FastText

# Преобразовываем текст в список списков слов (токенизация уже выполнена)
sentences = selected_columns['tweet'].apply(lambda x: x.split()).tolist()

#### a. Word2Vec (с использованием библиотеки Gensim)

In [8]:
# Word2Vec CBOW
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)
model.save("my_word2vec_model_cbow")

In [9]:
# Word2Vec Skip-gram
model = Word2Vec(sentences, vector_size=100, window=3, min_count=1, sg=1, epochs=3)
model.save("my_word2vec_model_sg")

#### b. FastText (с использованием библиотеки Gensim)

In [10]:
# FastText CBOW
model = FastText(sentences, vector_size=100, window=5, min_count=1, sg=0)
model.save("my_fasttext_model_cbow")

In [11]:
# FastText Skip-gram
model = FastText(sentences, vector_size=100, window=3, min_count=1, sg=1, epochs=3)
model.save("my_fasttext_model_sg")

In [12]:
# Обученная модель Word2Vec
word2vec_model = Word2Vec.load("my_word2vec_model_cbow")

# Обученная модель FastText
fasttext_model = FastText.load("my_fasttext_model_sg")

## Поиск ближайших твитов по запросу

In [13]:
def find_similar_tweets(query, dataset, num_results=5, model_type="word2vec"):
    # Проверяем, является ли query строкой, и преобразуйте в строку, если это необходимо
    if not isinstance(query, str):
        query = str(query)
    
    # Инициализация вектора запроса
    if model_type == "word2vec":
        model = word2vec_model
    elif model_type == "fasttext":
        model = fasttext_model
    else:
        raise ValueError("Неподдерживаемый тип модели. Допустимые значения: 'word2vec' и 'fasttext'.")
    
    query_vector_sum = np.zeros(model.vector_size)
    num_words = 0
    
    # Вычисление суммы векторов для слов в запросе
    for word in query.split():
        if word in model.wv:
            query_vector_sum += model.wv[word]
            num_words += 1
    
    # Если есть слова из запроса в модели, вычислите средний вектор
    if num_words > 0:
        query_vector = query_vector_sum / num_words
    else:
        # Если все слова из запроса отсутствуют в модели, верните None
        return None
    
    # Рассчет косинусного расстояния между запросом и всеми твитами
    similarities = []
    for tweet in dataset['tweet']:
        tweet_vector_sum = np.zeros(model.vector_size)
        num_tweet_words = 0
        for word in tweet.split():
            if word in model.wv:
                tweet_vector_sum += model.wv[word]
                num_tweet_words += 1
        if num_tweet_words > 0:
            tweet_vector = tweet_vector_sum / num_tweet_words
            cosine_similarity = np.dot(query_vector, tweet_vector) / (np.linalg.norm(query_vector) * np.linalg.norm(tweet_vector))
            similarities.append((tweet, cosine_similarity))
    
    # Сортировка твитов по близости к запросу
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    
    # Верните топ N ближайших твитов
    return sorted_similarities[:num_results]

### Проверка Word2Vec

In [14]:
query = "на улице осень"
num_results = 5
similar_tweets = find_similar_tweets(query, combined_df, num_results, model_type='word2vec')

print(f"Запрос: \"{query}\"")
print("Ближайшие твиты:")

for i, (tweet, similarity) in enumerate(similar_tweets, 1):
    print(f"{i}. Твит: {tweet}")
    print(f"   Сходство: {similarity:.4f}")
    print()

Запрос: "на улице осень"
Ближайшие твиты:
1. Твит: @A_Rainik Привет! Похоже не осень виновата,а Зимородки Созрели.)))
   Сходство: 1.0000

2. Твит: Солнечная осень в Ставрополе) http://t.co/a1UsljRR3u
   Сходство: 1.0000

3. Твит: @HZF0HB думала весной)но на март-апрель намечено завершение реформирования МСУ)не рискнут,ибо админресурс будет другим занят)ставлю на осень
   Сходство: 0.9967

4. Твит: RT @jufyzatugade: осень! мне осень всегда почему-то навевает эту песню   :) она такая грустная и добрая) но нравиццо!
   Сходство: 0.9940

5. Твит: RT @KulpanovichK: Пришла идея распечатать несколько фотографии с телефона и из инстаграма:) Намного приятнее их расматривать вживую)
   Сходство: 0.9836



### Проверка Fasttext

In [15]:
query = "на улице осень"
num_results = 5
similar_tweets = find_similar_tweets(query, combined_df, num_results, model_type='fasttext')

print(f"Запрос: \"{query}\"")
print("Ближайшие твиты:")

for i, (tweet, similarity) in enumerate(similar_tweets, 1):
    print(f"{i}. Твит: {tweet}")
    print(f"   Сходство: {similarity:.4f}")
    print()

Запрос: "на улице осень"
Ближайшие твиты:
1. Твит: Футбол, зимой, в армии, на плацу ахуено))
Я этого дожлался!)
   Сходство: 0.9988

2. Твит: радуемся снегу на нелюбимом 8 этаже)) #Киев #кпі http://t.co/4ULcbBN9QI
   Сходство: 0.9988

3. Твит: 2:46 на улице лютая метель :) http://t.co/jIYMOpZ70x
   Сходство: 0.9987

4. Твит: Эхх, на улице великолепно 8) http://t.co/t7nQGv9ech
   Сходство: 0.9987

5. Твит: #ЧитаюВзаимно погодка стоит прекрасная)))))))пошли на лыжи народ?))))))
   Сходство: 0.9986



## Вывод

На основе моих результатов анализа можно сделать следующие выводы и предположения:

1. **Word2Vec лучше подходит для задачи поиска похожих твитов**. Этот вывод можно сделать на того, что Word2Vec предоставляет более точные результаты для вашей конкретной задачи. Это может быть связано с различиями в архитектуре и обучении между Word2Vec и FastText.

2. **Различия в архитектуре**. Word2Vec и FastText имеют разные архитектуры. Word2Vec учитывает только слова как целостные сущности, в то время как FastText учитывает подслова (n-граммы). Это может сказываться на способности модели к выявлению семантической схожести между твитами.

3. **Комбинирование методов**. Возможно, для улучшения качества поиска похожих твитов можно попробовать комбинировать оба метода. Например, использовать Word2Vec для начального поиска, а затем уточнять результаты с помощью FastText.